# Multi-agent systems with Autogen

In [7]:
!pip install tiktoken==0.8.0


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\jorgegar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [22]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool,CodeInterpreterTool
import os
from dotenv import load_dotenv


In [23]:
# Load environment variables from .env file
load_dotenv()

True

## Orchestrate a single agent with Autogen

In [47]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4-turbo",
    model = "gpt-4-turbo",
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=token_provider,
)

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ.get("PROJECT_CONNECTION_STRING"),
)
bing_connection = project_client.connections.get(
    connection_name=os.environ.get("BING_CONNECTION_NAME")
)

conn_id = bing_connection.id
async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    with project_client:
        agent = project_client.agents.create_agent(
            model="gpt-4-turbo",
            name="my-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"User Message: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)
bing = BingGroundingTool(connection_id=conn_id)
async def assistant_run() -> None:
    response = await bing_search_agent.on_messages(
            [TextMessage(content="What is GitHub Copilot?", source="user")],
            cancellation_token=CancellationToken(),
    )
#     print(response.inner_messages)
    print(response.chat_message)
await assistant_run()

This is Bing for Azure AI Agent Service .......
Created agent, ID: asst_W6roMbcDn5GjqxUnOZY3xlru
Created thread, ID: thread_YlRdA9kdISDxrD7bDuoHftCi
User Message: {'id': 'msg_eJw4feoVGrgKANiwuxLPWgOm', 'object': 'thread.message', 'created_at': 1736942425, 'assistant_id': None, 'thread_id': 'thread_YlRdA9kdISDxrD7bDuoHftCi', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is GitHub Copilot?', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finished with status: completed
Deleted agent
Messages:GitHub Copilot is an artificial intelligence (AI) code completion tool designed to help programmers write code faster and more efficiently. It was developed by GitHub in collaboration with OpenAI, the creators of ChatGPT and DALL-E. Introduced initially as a limited technical preview in October 2021, it became widely available to all developers in June 2022 through a subscription service【3†source】.

The tool functions as a coding assistant, offerin

## Orchestrate several agents with Autogen

In [50]:
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

az_model_client  = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4-turbo",
    model = "gpt-4-turbo",
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=token_provider,
)
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ.get("PROJECT_CONNECTION_STRING"),
)
bing_connection = project_client.connections.get(
    connection_name=os.environ.get("BING_CONNECTION_NAME")
)

conn_id = bing_connection.id


In [51]:
async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    # with project_client:
    agent = project_client.agents.create_agent(
            model="gpt-4-turbo",
            name="my-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
    )
    print(f"SMS: {message}")
        # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

        # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])

        # project_client.close()

    return messages["data"][0]["content"][0]["text"]["value"]



In [52]:
async def save_blog_agent(blog_content: str) -> str:

    print("This is Code Interpreter for Azure AI Agent Service .......")


    code_interpreter = CodeInterpreterTool()
        
    agent = project_client.agents.create_agent(
            model="gpt-4-turbo",
            name="my-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

        # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
        # project_client.agents.delete_file(file.id)
        # print("Deleted file")

        # print the messages from the agent
    messages = project_client.agents.get_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

        # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

        # print(f"File: {messages.file_path_annotations}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")


        # project_client.close()


    return "Saved"


    
 

In [53]:
   
bing_search_agent = AssistantAgent(
    name="bing_search_agent",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="You are a search expert, help me use tools to find relevant knowledge",
)
save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when your blog are saved.
    """
)
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination
reflection_team = RoundRobinGroupChat([bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)
await Console(
    reflection_team.run_stream(task="""

                    I am writing a blog about machine learning. Search for the following 3 questions and write a blog based on the search results ,save it
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                               

    """)
)  # Stream the messages to the console.

This is Bing for Azure AI Agent Service .......
Created agent, ID: asst_JLqHKJfs4E55Rj8r4HgmbW49
Created thread, ID: thread_XalVMTh03QbNpgBtKECmv1Ek
SMS: {'id': 'msg_KBTOrpImZXxVvEOp1KgIWZUB', 'object': 'thread.message', 'created_at': 1736942837, 'assistant_id': None, 'thread_id': 'thread_XalVMTh03QbNpgBtKECmv1Ek', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Machine Learning?', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finished with status: completed
Deleted agent
Messages:Machine learning is a subfield of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks effectively without using explicit instructions. Instead, these systems learn and make decisions based on data.

In machine learning, algorithms build a model based on sample data, known as "training data," in order to make predictions or decisions without being explicitly progr

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='\n\n                    I am writing a blog about machine learning. Search for the following 3 questions and write a blog based on the search results ,save it\n                    \n                    1. What is Machine Learning?\n                    2. The difference between AI and ML\n                    3. The history of Machine Learning\n                               \n\n    ', type='TextMessage'), ToolCallRequestEvent(source='bing_search_agent', models_usage=RequestUsage(prompt_tokens=122, completion_tokens=19), content=[FunctionCall(id='call_f216DzFLPByYo5r12g7YCgAr', arguments='{"query":"What is Machine Learning?"}', name='web_ai_agent')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='bing_search_agent', models_usage=None, content=[FunctionExecutionResult(content='Machine learning is a subfield of artificial intelligence (AI) that focuses on the development of algorithms and statistical 